In [1]:
import pandas as pd

In [2]:
audiocaps = {
    "train": pd.read_csv("../data/audiocaps/csvs/train.csv"),
    "valid": pd.read_csv("../data/audiocaps/csvs/valid.csv"),
    "test": pd.read_csv("../data/audiocaps/csvs/test.csv"),
}

for name, split in audiocaps.items():
    print(name, len(split), "unique:",  len(split["youtube_id"].unique()))


train 49838 unique: 49838
valid 2475 unique: 495
test 4875 unique: 975


In [3]:
audiocaps_all = pd.concat(audiocaps.values())
len(audiocaps_all)

57188

In [4]:
audiocaps_all.columns

Index(['audiocap_id', 'youtube_id', 'start_time', 'caption'], dtype='object')

In [5]:
import os
exists = audiocaps_all["youtube_id"].drop_duplicates().apply(lambda x: os.path.exists("../data/audioset_full/audios/train_unbalanced/" + x + ".mp3"))

In [6]:
exists.mean()

0.9107351680049894

In [7]:
exists_2 = audiocaps_all["youtube_id"].drop_duplicates().apply(lambda x: os.path.exists("../data/audioset_full/audios/train_balanced/" + x + ".mp3"))
exists_3 = audiocaps_all["youtube_id"].drop_duplicates().apply(lambda x: os.path.exists("../data/audioset_full/audios/eval/" + x + ".mp3"))

In [8]:
exists.sum() == (exists | exists_2 | exists_3).sum()

True

In [9]:
audioset_small = {
    "train": pd.read_json("../data/audioset_small/annotations/train.jsonl", lines=True),
    "valid": pd.read_json("../data/audioset_small/annotations/valid.jsonl", lines=True),
    "test": pd.read_json("../data/audioset_small/annotations/test.jsonl", lines=True),
}

audioset_small_all = pd.concat(audioset_small.values())

In [10]:
splits = {"train", "valid", "test"}

# CHECK LEAKAGE

for split in splits:
    for other_split in splits - {split}:
        assert set.isdisjoint(set(audioset_small[split]["youtube_id"]), set(audioset_small[other_split]["youtube_id"]))
        assert set.isdisjoint(set(audiocaps[split]["youtube_id"]), set(audiocaps[other_split]["youtube_id"]))
        assert set.isdisjoint(set(audioset_small[split]["youtube_id"]), set(audiocaps[other_split]["youtube_id"]))


In [20]:
audiocaps["valid"].columns

Index(['audiocap_id', 'youtube_id', 'start_time', 'caption'], dtype='object')

In [43]:
(
    audiocaps["valid"]
    .drop(columns=["audiocap_id", "start_time"])
    .groupby("youtube_id")
    .agg(list)["caption"]
    .apply(pd.Series)
    .rename(columns=lambda x: "caption_" + str(x+1))
    .reset_index()
)

,youtube_id,caption_1,caption_2,caption_3,caption_4,caption_5
0,rqfQRErjfk8,A large crowd cheers and applauds,An audience screams and gives applause,Continuous applause and cheering,An audience screams and gives applause,An audience cheers
1,rqu8iB22I_Y,Popping and crackling repeats as men yell and ...,A vehicle is running and crackling and popping...,People yell and laugh as a motor sputters and ...,Men screeching and shouting while a vehicle is...,People yell and laugh as a vehicle approaches ...
2,rwT__ERCUno,An engine idles consistently before sputtering...,A motorcycle engine is idling,A sputtering motor idles roughly,Sputtering and clanking of an idling powerful ...,An engine is idling while people speak
3,rwb6PSAee5Y,A toilet flushes quickly,A toilet flushes loudly and water runs down th...,A toilet flushes and water drains,A loud rumble followed by a toilet flushing,A toilet flushes and water drains
4,rwtmaKiCcQU,Distant men speak as a spray can nozzle is dep...,"While a man talks in the distance, a sprayer s...",Intermittent spraying is accompanied by men sp...,A spray hisses followed by a man speaking and ...,A person sprays liquid onto a hard surface as ...
...,...,...,...,...,...,...
490,ztQ79sWWC34,A child speaks with some light rustling and cl...,A young child speaks and moves a spoon in dishes,A girl talks while objects hit,A child speaks and uses dishes,"A child is speaking, shuffling and faint scrap..."
491,zuua6-5goWw,Birds chirp and a pop occurs before a man speaks,Birds chirp quietly and an adult man speaks,A soft plastic clack as seagulls squawk in the...,Birds chirp and a man speaks,Birds twitter and chirp as a man speak quietly
492,zvABdOhz9fE,A man talks while some wood is rubbed far away,A man speaking as wood repeatedly clacks in th...,A man is talking loudly while another man spok...,A man speaking continuously with some clanking,A man speaking with some clanking
493,zvmXKyAFq80,A vehicle engine is revving the idles,A rough engine revs and sputters,Humming and rattling of an idling and revving ...,A car revs loudly a few times while idling,A vehicle is revving and grooming
